In [ ]:
# 导入需要用到的模块
## 数值计算模块numpy。
import numpy as np
## 数据处理模块pandas。
import pandas as pd

In [ ]:
# 数据利用

## 读取数据
### 从工作目录下的'data.xls'文件中的'data'表中读取数据，没有索引列，空值用'NA'填充。请确认文件路径、文件名、表名正确。
data = pd.read_excel('data-done.xls', 'data', index_col=None, na_values=['NA'])
### 输出读取数据的数量
print('数据量为： ' + str(len(data)) + ' 行。')

In [ ]:
# 以下为生成报表，包括街镇报表、职业报表、年龄报表、周累计报表、月累计报表、动态分析表。

In [ ]:
## 筛选数据，筛选出拟分析的一个时间段内的数据：2020年第18周的数据。可手动修改，比如选取某月的数据。
### 定义起止时间点
data_done_start = pd.Timestamp('2020-05-04')
data_done_end = pd.Timestamp('2020-05-11')
### 选取数据
data_done = data[(data['订正终审时间'] >= data_done_start) & (data['订正终审时间'] < data_done_end)]
### 输入选取数据的量
print('筛选出数据量为： ' + str(len(data_done)) + ' 行。')

In [ ]:
## 查看甲乙类传染病“疾病分组2”的值域及数量。选择“疾病分组1”列值为甲、乙类传染病的记录，对“疾病分组2”列进行分类汇总。
data_done[data_done['疾病分组1'].isin(['甲类传染病','乙类传染病'])]['疾病分组2'].value_counts()

In [ ]:
## 街镇报表

### 读取辅助表“街镇”。
jiezhen = pd.read_excel('辅助表.xls', '街镇', index_col=None, na_values=['NA'])

### 数据透视。“街镇分组”*（“疾病分组3”、“疾病名称2”）数据透视。
table_jiezhen = pd.pivot_table(data_done, values = '疾病名称', index = ['现住地址国标', '街镇分组'], columns = ['疾病分组3', '疾病名称2'], aggfunc = 'count', fill_value = 0, margins = 1)
befor = table_jiezhen['All'].sum()
### 完善报表，对“街镇分组”重新索引。数据透视中，合计为0的街镇，默认不放入透视表中。重新索引，实现了插入合计为0街镇的目的。
table_jiezhen = table_jiezhen.reindex([jiezhen['现住地址国标'],jiezhen['街镇分组']]).fillna(value=0)
after = table_jiezhen['All'].sum()
### 校验重新索引是否有误。重新索引时，删除了“合计”行，“合计”列的合计值减半。
if(after == befor / 2) : print('重新索引无误。')
    
### 保存街镇报表
table_jiezhen.to_excel('result/街镇报表.xls')
table_jiezhen

In [ ]:
## 职业报表

### 读取辅助表“职业报表”。
baobiao_zhiye = pd.read_excel('辅助表.xls', '职业报表', index_col=None, na_values=['NA'])

#### 选取“疾病名称2”列值为“肺结核”的数据。
data_done_zhiye = data_done[(data_done['疾病名称2'] == '肺结核')]
#### 计算各职业、“户籍”值为“本市”的数据行数，赋值给职业报表的“肺结核-本市”列。
baobiao_zhiye['肺结核-本市'] = baobiao_zhiye['人群分类2'].apply(lambda x:len(data_done_zhiye[(data_done_zhiye['人群分类2'] == x) & (data_done_zhiye['户籍'] == '本市')]))
#### 计算各职业、“户籍”值为“外来”的数据行数，赋值给职业报表的“肺结核-外来”列。
baobiao_zhiye['肺结核-外来'] = baobiao_zhiye['人群分类2'].apply(lambda x:len(data_done_zhiye[(data_done_zhiye['人群分类2'] == x) & (data_done_zhiye['户籍'] == '外来')]))
#### “肺结核-本市”列值加上“肺结核-外来”列值，赋值给职业报表的“肺结核-合计”列。
baobiao_zhiye['肺结核-合计'] = baobiao_zhiye['肺结核-本市'] + baobiao_zhiye['肺结核-外来']

### 列出其他需要纳入报表的疾病名称，或者set(data_done['疾病名称'])，对所有的“疾病名称”进行汇总。
jibingList = ['猩 红 热', '手足口病', '水痘', '流行性感冒']
#### jibing = set(data_done['疾病名称'])

### 处理疾病列表中的每一个疾病
for i in jibingList :
    #### 选取“疾病名称”列值为各疾病名称的数据。
    data_done_zhiye = data_done[(data_done['疾病名称'] == i)]
    #### 计算各职业、“户籍”值为“本市”的数据行数，赋值给职业报表的“疾病名称-本市”列。
    baobiao_zhiye[i + '-本市'] = baobiao_zhiye['人群分类2'].apply(lambda x:len(data_done_zhiye[(data_done_zhiye['人群分类2'] == x) & (data_done_zhiye['户籍'] == '本市')]))
    #### 计算各职业、“户籍”值为“外来”的数据行数，赋值给职业报表的“疾病名称-外来”列。
    baobiao_zhiye[i + '-外来'] = baobiao_zhiye['人群分类2'].apply(lambda x:len(data_done_zhiye[(data_done_zhiye['人群分类2'] == x) & (data_done_zhiye['户籍'] == '外来')]))
    #### “疾病名称-本市”列值加上“疾病名称-外来”列值，赋值给职业报表的“疾病名称-合计”列。
    baobiao_zhiye[i + '-合计'] = baobiao_zhiye[i + '-本市'] + baobiao_zhiye[i + '-外来']
    
### 保存职业报表
baobiao_zhiye.to_excel('result/职业报表.xls')
baobiao_zhiye

In [ ]:
## 年龄报表

### 读取辅助表“年龄报表”。
baobiao_age = pd.read_excel('辅助表.xls', '年龄报表', index_col=None, na_values=['NA'])

### 列出疾病，或者set(data_done['疾病名称'])，对所有的“疾病名称”进行汇总。
jibingList = ['猩 红 热', '手足口病', '水痘', '流行性感冒']
#### jibing = set(data_done['疾病名称'])

### 处理疾病列表中的每一个疾病
for i in jibingList :
    #### 选取“疾病名称”列值为各疾病名称的数据。
    data_done_age = data_done[(data_done['疾病名称'] == i)]
    #### 计算各年龄组、“户籍”值为“本市”的数据行数，赋值给年龄报表的“疾病名称-本市”列。
    baobiao_age[i + '-本市'] = baobiao_age['年龄分组'].apply(lambda x:len(data_done_age[(data_done_age['年龄分组'] == x) & (data_done_age['户籍'] == '本市')]))
    #### 计算各年龄组、“户籍”值为“外来”的数据行数，赋值给年龄报表的“疾病名称-外来”列。
    baobiao_age[i + '-外来'] = baobiao_age['年龄分组'].apply(lambda x:len(data_done_age[(data_done_age['年龄分组'] == x) & (data_done_age['户籍'] == '外来')]))
    #### “疾病名称-本市”列值加上“疾病名称-外来”列值，赋值给年龄报表的“疾病名称-合计”列。
    baobiao_age[i + '-合计'] = baobiao_age[i + '-本市'] + baobiao_age[i + '-外来']

### 保存年龄报表
baobiao_age.to_excel('result/年龄报表.xls')
baobiao_age

In [ ]:
##  周累计报表

### 设置周数。
dangqianzhounum = 18
### 选取2020全年的数据。2020年第1周始于2019年12月30日，2020年最后一周第53周终于2021年01月03日。
data_zhou = data[(data['订正终审时间'] >= pd.Timestamp('2019-12-30')) & (data['订正终审时间'] < pd.Timestamp('2021-01-04'))]
### 生成周累计报表“周数”列
baobiao_zhou = pd.DataFrame({'周数' : np.arange(1, dangqianzhounum + 1)})

### 列出疾病列表，或者set(data_done['疾病名称'])，对所有的“疾病名称”进行汇总。
jibingList = ['手足口病', '麻 疹', '猩 红 热', '水痘', '流行性感冒']
#### jibingList = set(data_done['疾病名称'])

### 处理疾病列表中的每一个疾病
for i in jibingList :
    #### 选取“疾病名称”列值为各疾病名称的数据
    data_zhou_jibing = data_zhou[data_zhou['疾病名称'] == i]
    #### 计算各周、“户籍”值为“本市”的数据行数，赋值给周累计报表的“疾病名称-本市”列。
    baobiao_zhou[i + '-本市'] = baobiao_zhou['周数'].apply(lambda x:len(data_zhou_jibing[(data_zhou_jibing['周-终审'] == x) & (data_zhou_jibing['户籍'] == '本市')]))
    #### 计算各周、“户籍”值为“外来”的数据行数，赋值给周累计报表的“疾病名称外来”列。
    baobiao_zhou[i + '-外来'] = baobiao_zhou['周数'].apply(lambda x:len(data_zhou_jibing[(data_zhou_jibing['周-终审'] == x) & (data_zhou_jibing['户籍'] == '外来')]))
    #### “疾病名称-本市”列值加上“疾病名称-外来”列值，赋值给周累计报表的“疾病名称-合计”列。
    baobiao_zhou[i + '-合计']= baobiao_zhou[ i + '-本市'] + baobiao_zhou[i + '-外来']
    
### 保存报表
baobiao_zhou.to_excel('result/周累计报表.xls')
baobiao_zhou

In [ ]:
## 月累计报表

### 设置月数。
dangqianyuenum = 5

### 选取2020全年的数据，这里和“周累计”中全年数据的意义略有不同。
data_yue = data[(data['订正终审时间'] >= pd.Timestamp('2020-01-01')) & (data['订正终审时间'] < pd.Timestamp('2021-01-01'))]
### 生成月累计报表“月数”列
baobiao_yue = pd.DataFrame({'月数' : np.arange(1, dangqianyuenum + 1)})

### 列出疾病列表，或者set(data_done['疾病名称'])，对所有的“疾病名称”进行汇总。
jibingList = ['手足口病', '麻 疹', '猩 红 热', '水痘', '流行性感冒']
#### jibingList = set(data_done['疾病名称'])

### 处理疾病列表中的每一个疾病
for i in jibingList :
    #### 选取“疾病名称”列值为“手足口病”的数据
    data_yue_jibing = data_yue[data_yue['疾病名称'] == i]
    #### 计算各月、“户籍”值为“本市”的数据行数，赋值给月累计报表的“疾病名称-本市”列。
    baobiao_yue[i + '-本市'] = baobiao_yue['月数'].apply(lambda x:len(data_yue_jibing[(data_yue_jibing['月-终审'] == x) & (data_yue_jibing['户籍'] == '本市')]))
    #### 计算各月、“户籍”值为“外来”的数据行数，赋值给月累计报表的“疾病名称-外来”列。
    baobiao_yue[i + '-外来'] = baobiao_yue['月数'].apply(lambda x:len(data_yue_jibing[(data_yue_jibing['月-终审'] == x) & (data_yue_jibing['户籍'] == '外来')]))
    #### “疾病名称-本市”列值加上“疾病名称-外来”列值，赋值给月累计报表的“疾病名称-合计”列。
    baobiao_yue[i + '-合计']= baobiao_yue[i + '-本市'] + baobiao_yue[i + '-外来']

### 保存报表
baobiao_yue.to_excel('result/月累计报表.xls')
baobiao_yue

In [ ]:
## 月度动态分析表。

year = 2020 #### 年份值，需手动输入
month = 4   #### 月份值，需手动输入

### 读取辅助表中的“月报1”、“月报2”，“月报2”是对“肝 炎”、“肺结核”、“梅毒”三种疾病详细描述，比如，“肝炎”分为“甲肝”、“乙肝”、“丙肝”、“戊肝”、“未分型”五种。
baobiao_yue1 = pd.read_excel('辅助表.xls', '月报1', index_col=None, na_values=['NA'])
baobiao_yue2 = pd.read_excel('辅助表.xls', '月报2', index_col=None, na_values=['NA'])


### 计算时间值：本月起始时间、上月起始时间、去年同期起始时间、本年累计起始时间、去年同期累计起始时间。
#### 本月起始时间
benyueStart = str(year) + '-' + str(month) + '-01'
benyueEnd = str(year) + '-' + str(month + 1) + '-01'
#### 上月起始时间
if month == 1 :
    shangyueStart = str(year-1) + '-12-01'
    shangyueEnd = str(year) + '-01-01'
else:
    shangyueStart = str(year) + '-' + str(month - 1) + '-01'
    shangyueEnd = benyueStart
#### 去年同期起始时间
qunianStart = str(year - 1) + '-' + str(month) + '-01'
qunianEnd = str(year - 1) + '-' + str(month + 1) + '-01'
#### 本年累计起始时间
bennianleijiStart = str(year) + '-01-01'
bennianleijiEnd = benyueEnd 
#### 去年同期累计起始时间
qunianleijiStart = str(year - 1) + '-01-01'
qunianleijiEnd = qunianEnd

### 数据汇总，大部分是重复的代码，未进行优化。

#### 选择本月-4月份数据
data_benyue = data[(data['订正终审时间'] >= pd.Timestamp(benyueStart)) & (data['订正终审时间'] < pd.Timestamp(benyueEnd))]
#### 计算“疾病名称2”列值为月度动态分析表1中“病名”列值的数据行数，赋值给月度动态分析表1的“本月”列。月度动态分析表被分成了两部分，详见辅助表，完成后进行合并。
baobiao_yue1['本月'] = baobiao_yue1['病名'].apply(lambda x : len(data_benyue[(data_benyue['疾病名称2'] == x)]))
#### 计算“疾病名称1”列值为月度动态分析表2中“病名”列值的数据行数，赋值给月度动态分析表2的“本月”列。
baobiao_yue2['本月']= baobiao_yue2['病名'].apply(lambda x : len(data_benyue[(data_benyue['疾病名称1'] == x)]))

### 汇总“上月”数据
data_shangyue = data[(data['订正终审时间'] >= pd.Timestamp(shangyueStart)) & (data['订正终审时间'] < pd.Timestamp(shangyueEnd))]
baobiao_yue1['上月'] = baobiao_yue1['病名'].apply(lambda x : len(data_shangyue[(data_shangyue['疾病名称2'] == x)]))
baobiao_yue2['上月'] = baobiao_yue2['病名'].apply(lambda x : len(data_shangyue[(data_shangyue['疾病名称1'] == x)]))
baobiao_yue1['本月与上月相比'] = round((baobiao_yue1['本月'] / baobiao_yue1['上月'] - 1) * 100, 2)
baobiao_yue2['本月与上月相比'] = round((baobiao_yue2['本月'] / baobiao_yue2['上月'] - 1) * 100, 2)

### 汇总“去年同期”数据
data_quniantongqi = data[(data['订正终审时间'] >= pd.Timestamp(qunianStart)) & (data['订正终审时间'] < pd.Timestamp(qunianEnd))]
baobiao_yue1['去年同期']= baobiao_yue1['病名'].apply(lambda x:len(data_quniantongqi[(data_quniantongqi['疾病名称2'] == x)]))
baobiao_yue2['去年同期']= baobiao_yue2['病名'].apply(lambda x:len(data_quniantongqi[(data_quniantongqi['疾病名称1'] == x)]))
baobiao_yue1['本月与去年同期相比'] = round((baobiao_yue1['本月'] / baobiao_yue1['去年同期'] - 1) * 100, 2)
baobiao_yue2['本月与去年同期相比'] = round((baobiao_yue2['本月'] / baobiao_yue2['去年同期'] - 1) * 100, 2)

### 汇总“本年累计”数据
data_bennian = data[(data['订正终审时间'] >= pd.Timestamp(bennianleijiStart)) & (data['订正终审时间'] < pd.Timestamp(bennianleijiEnd))]
baobiao_yue1['本年累计']= baobiao_yue1['病名'].apply(lambda x:len(data_bennian[(data_bennian['疾病名称2'] == x)]))
baobiao_yue2['本年累计']= baobiao_yue2['病名'].apply(lambda x:len(data_bennian[(data_bennian['疾病名称1'] == x)]))

### 汇总“去年同期累计”数据
data_quniantongqileiji = data[(data['订正终审时间'] >= pd.Timestamp(qunianleijiStart)) & (data['订正终审时间'] < pd.Timestamp(qunianleijiEnd))]
baobiao_yue1['去年同期累计']= baobiao_yue1['病名'].apply(lambda x:len(data_quniantongqileiji[(data_quniantongqileiji['疾病名称2'] == x)]))
baobiao_yue2['去年同期累计']= baobiao_yue2['病名'].apply(lambda x:len(data_quniantongqileiji[(data_quniantongqileiji['疾病名称1'] == x)]))
baobiao_yue1['本年累计与去年同期累计相比'] = round((baobiao_yue1['本年累计'] / baobiao_yue1['去年同期累计'] - 1) * 100, 2)
baobiao_yue2['本年累计与去年同期累计相比'] = round((baobiao_yue2['本年累计'] / baobiao_yue2['去年同期累计'] - 1) * 100, 2)

### 合并月度动态分析表。
baobiao_yue=baobiao_yue1.append(baobiao_yue2)

### 保存月度动态分析表
baobiao_yue.to_excel('result/月度动态分析表.xls')
baobiao_yue